# Chatcompletion Pro（对话）

本接口为 abab5.5 模型的调用接口。在 abab5.5 模型下，我们强烈建议您使用本接口，因为它提供了 chat completion 接口无法满足的功能。

相比于 chat completion 接口，本接口更新了请求体和返回体的结构，以更好地适配多人多bot对话场景。此外，还增加了以下四个功能：

1. **示例对话功能（Sample Message）**：通过示例对话，帮助模型更好地理解用户想要获得的返回信息，包括但不限于信息内容、信息格式和信息回复方式等。

2. **限制返回格式（Glyph）**：帮助用户强制要求模型按照配置的固定格式返回内容。当前支持的配置格式详见下表。

3. **函数调用（Function Call）**：可调用自有函数生成内容，辅助大模型生成结果。这可以帮助模型获取用户希望获得的相应信息，从而使模型返回更符合用户个人实际情况的内容。

4. **插件功能（Plugin）**：目前支持调用我们集成的搜索引擎生成内容，提供网络引擎检索的内容，辅助大模型生成结果。

接口本身是无状态的，即单次调用时，模型只接收传入内容，不涉及业务逻辑，同时模型也不存储您传入的数据。

如果输入内容或输出内容严重违规，接口会返回内容违规错误信息，并回复内容为空。

## 场景类型

| 描述       | 举例                                |
|------------|-----------------------------------|
| 人设对话   | 让大模型扮演多个人物，每个人物有姓名、性别、特别性格特点、一定背景设定，可以支持多人物与您、多个人物之间进行闲聊对话。剧本撰写灵感提供、虚拟情感陪伴闲聊、多角色互动娱乐、动漫/游戏人物互动演绎。 |
| 生产力     | 让大模型当作一个有一定知识领域的机器人来回答问题。知识问答、总结说明、情感分析、文本分类、大纲&营销文案生成、作文批改等实时信息检索、信息提取、规定格式输出。 |


## API

- **API地址**: [https://api.minimax.chat/v1/text/chatcompletion_pro](https://api.minimax.chat/v1/text/chatcompletion_pro)

## 支持token数

| 模型名称     | 最大token数（一次请求+回复的输入输出总token） |
|--------------|-------------------------------------------|
| abab5.5-chat | 16384                                     |


In [ ]:
import requests

group_id="请填写您的group_id"
api_key="请填写您的api_key"

url = "https://api.minimax.chat/v1/text/chatcompletion_pro?GroupId=" + group_id

payload = {
    "bot_setting": [
        {
            "bot_name": "MM智能助理",
            "content": "MM智能助理是一款由MiniMax自研的，没有调用其他产品的接口的大型语言模型。MiniMax是一家中国科技公司，一直致力于进行大模型相关的研究。",
        }
    ],
    "messages": [{"sender_type": "USER", "sender_name": "小明", "text": "帮我用英文翻译下面这句话：我是谁"}],
    "reply_constraints": {"sender_type": "BOT", "sender_name": "MM智能助理"},
    "model": "abab5.5-chat",
    "tokens_to_generate": 1034,
    "temperature": 0.01,
    "top_p": 0.95,
}
headers = {"Content-Type": "application/json", "Authorization": "Bearer " + api_key}

response = requests.request("POST", url, headers=headers, json=payload)

print(response.status_code)
print(response.text)

## 2. 示例对话功能（Sample Messages）

### 功能说明

该功能可以帮助模型更好地理解用户想要获得的返回信息，包括但不限于信息内容、信息格式、信息回复方式等。

### 请求体（Request）参数

如果要使用此功能，除了使用基础功能参数之外，还需要额外添加以下参数。

| 参数位置           | 参数                  | 语义             | 是否必填 | 取值                           |
|--------------------|-----------------------|------------------|----------|--------------------------------|
| body               | sample_messages       | 示例对话内容       | 否       | list                         |
| sample_messages.sender_type | 发送者的类型     | 是       | string（需要为以下三个合法值之一）：USER（用户发送的内容）、BOT（模型生成的内容）、FUNCTION（详见下文中的函数调用部分） |
| sample_messages.sender_name | 发送者的名字     | 是       | string                         |
| sample_messages.text       | 消息内容       | 是       | string                         |


In [ ]:
{
    "bot_setting": [ 
        {
            "bot_name": "MM智能助理",
            "content": "MM智能助理是一款由MiniMax自研的，没有调用其他产品的接口的大型语言模型。MiniMax是一家中国科技公司，一直致力于进行大模型相关的研究。"
        }
    ],
    "sample_messages": [
        {
            "sender_type": "USER",
            "sender_name": "小明",
            "text": "把A5处理成红色，并修改内容为minimax"
        },
        {
            "sender_type": "BOT",
            "sender_name": "MM智能助理",
            "text": "select A5 color red change minimax"
        }
    ],
    "messages": [ 
        {
            "sender_type": "USER",
            "sender_name": "小明",
            "text": "你会按照如下要求回复我的内容:需要根据以下接口列表，对我给定的内容给出调用接口的顺序和参数。你只需要给出接口的调用顺序和参数，除此之外不要给出任何其他的输出。以下是可用的接口列表:select: 选择具体的表格位置，输入参数使用字母和数字确定，例如B13”;color: 对选定的表格位置染色，输入参数使用颜色的英文名，例如“red”;change: 对选定的表格位置进行修改，输入参数使用字符串。"
        },
        {
    "sender_type": "USER",
    "sender_name": "小明",
    "text": "把B6处理成灰色，修改内容为question"
        }
    ],
    "reply_constraints": { 
        "sender_type": "BOT",
        "sender_name": "MM智能助理"
    },
    "model": "abab5.5-chat", 
    "tokens_to_generate": 1034,
    "temperature": 0.01,
    "top_p": 0.95
}

## 4.3. 限制返回格式功能（Glyph）

### 功能说明

该功能可以帮助用户强制要求模型按照配置的固定格式返回内容，当前支持的配置格式详见下表。

举例：

如果 glyph 为 "《蒙娜丽莎》是{{gen 'name'}}的画作"
模型可能会输出 "《蒙娜丽莎》是列奥纳多·达·芬奇的画作"

模型不会输出 "列奥纳多·达·芬奇画了《蒙娜丽莎》"



如果 glyph 为 "西柚的类型为{{select 'type' options=['家电','其他']}}"
模型可能会输出 "西柚的类型为其他"

模型不会输出 "西柚的类型为水果"

### 请求体（Request）参数

如果要使用此功能，除了使用基础功能参数之外，还需要额外添加以下参数。

| 参数位置                           | 参数                           | 语义           | 取值     | 备注                              |
|------------------------------------|--------------------------------|----------------|----------|-----------------------------------|
| body                               | reply_constraints.glyph        | 限制的返回格式配置 | dict     |                                   |
| reply_constraints.glyph.type       | 使用什么模板功能当前仅支持     | 1、raw<br>2、json_value | string   |                                   |
| reply_constraints.glyph.raw_glpyh  | 需要限制的格式要求，使用 glpyh 语法 | string   | 详见下文 glyph 语法说明章节     |
| reply_constraints.glyph.json_properties | 需要限制的格式要求，使用 dic 描述 JSON 语法 | dict   | 包括 properties，properties 包括参数及每个参数的类型 type。包含三种基础类型：string、boolean、number 以及对象类型，支持嵌套：object。可以参考下面的 JSON value 的请求事例。 |


In [ ]:
{
    "bot_setting": [ 
        {
            "bot_name": "MM智能助理",
            "content": "MM智能助理是一款由MiniMax自研的，没有调用其他产品的接口的大型语言模型。MiniMax是一家中国科技公司，一直致力于进行大模型相关的研究。"
        }
    ],
    "messages": [ 
        {
            "sender_type": "USER",
            "sender_name": "小明",
            "text": "请用英语帮我翻译下面这句话：我是谁"
        }
    ],
    "reply_constraints": { 
        "sender_type": "BOT",
        "sender_name": "MM智能助理",
        "glyph": {
            "type": "raw",
            "raw_glyph": "这句话的翻译是：{{gen 'content'}}"
        }
    },
    "model": "abab5.5-chat", 
    "tokens_to_generate": 1034,
    "temperature": 0.01,
    "top_p": 0.95
}

## 4.4 函数调用功能（Functions）

### 功能说明

该功能可调用自有函数生成内容，辅助大模型生成结果。可以帮助模型获取用户希望模型获取的相应信息，以使得模型返回更符合用户个人实际情况的内容。

### 请求体（Request）参数

如果要使用此功能，除了使用基础功能参数之外，还需要额外添加以下参数。

| 参数位置         | 参数                | 语义                                           | 取值                        | 备注                                       |
|------------------|---------------------|------------------------------------------------|-----------------------------|--------------------------------------------|
| body             | functions            | 模型可以为其生成 JSON 输入的函数列表             | list                        |                                            |
| functions.name   | 函数名称              | 需要满足 [a-zA-Z0-9-_]{1,64} 格式，并且不能以 plugin_ 开头 | string                      |                                            |
| functions.description | 函数的功能说明 | 请在说明中明确函数的功能，会作为模型是否调用函数的参考 | string                      |                                            |
| functions.parameters | 该函数的参数及其说明 | 包括 properties 以及 required，properties 包括参数及每个参数的类型 type 和描述，required 代表必须要返回的参数 | dict |                                            |


In [ ]:
import requests
import json

group_id="请填写您的group_id"
api_key="请填写您的api_key"

# 1. 定义自己的function实现


def ChatHistory_search(query: str) -> str:
    # 待补充具体代码，如从数据库中获取聊天记录
    return """
用户 下午9：09：57
你昨天去做什么了

小白 下午9：10：07 
没做什么，去新开的书店逛了逛。

用户 下午9：10：10 
8月19日京都有烟花大会，我要去！

小白 下午9：10：16 
哇！真的吗，我也想去，要一起吗？

用户 下午9：10：19 
好呀，那我们到时候见！

小白 下午9：10：21
嗯嗯，听说那边有个甜品也很不错，我们可以一起尝尝～
"""


# 2. 使用functions选项调用ccp接口

url = f"https://api.minimax.chat/v1/text/chatcompletion_pro?GroupId={group_id}"
headers = {"Authorization": f"Bearer {api_key}", "Content-Type": "application/json"}
payload = {
    "bot_setting": [
        {
            "bot_name": "MM智能助理",
            "content": "MM智能助理是一款由MiniMax自研的，没有调用其他产品的接口的大型语言模型。MiniMax是一家中国科技公司，一直致力于进行大模型相关的研究。",
        }
    ],
    "reply_constraints": {"sender_type": "BOT", "sender_name": "MM智能助理"},
    "plugins": [],
    "functions": [
        {
            "name": "ChatHistory_search",
            "description": "搜索给定聊天记录，总结并返回相关结果",
            "parameters": {
                "type": "object",
                "properties": {"query": {"type": "string", "description": "要搜索的内容关键词"}},
                "required": ["query"],
            },
        }
    ],
    "sample_messages": [],
    "messages": [
        {"sender_type": "USER", "sender_name": "用户", "text": "我昨天说8月19日要去哪里来着？"}
    ],
    "model": "abab5.5-chat",
    "tokens_to_generate": 1000,
    "temperature": 0.01,
    "top_p": 0.95,
    "stream": False,
}

response = requests.post(url, headers=headers, json=payload)

print(response.status_code)
print(response.text)

# 3. 解析ccp接口的响应，并调用自己的function

assert response.status_code == 200
json_resp = response.json()
assert json_resp["base_resp"]["status_code"] == 0
assert json_resp["function_call"]["name"] == "ChatHistory_search"
assert json_resp["function_call"]["arguments"] != ""

arguments = json.loads(json_resp["function_call"]["arguments"])
func_resp = ChatHistory_search(arguments["query"])
print(func_resp)

# 4. 将自己的function的响应结果返回给ccp接口
payload["messages"].extend(json_resp["choices"][0]["messages"])
payload["messages"].append(
    {"sender_type": "FUNCTION", "sender_name": "MM智能助理", "text": func_resp}
)

response = requests.post(url, headers=headers, json=payload)

print(response.status_code)
print(response.text)

    

## 4.5. 插件功能（Plugins）

### 功能说明

该功能目前支持调用我们集成的搜索引擎生成内容，提供网络引擎检索的内容，辅助大模型生成结果。

### 请求体（Request）参数

如果要使用此功能，除了使用基础功能参数之外，还需要额外添加以下参数。


In [ ]:
model": "abab5.5-chat",
    "stream": false,
    "tokens_to_generate": 1024,
    "messages": [
        {
            "sender_type": "USER",
            "sender_name": "小明",
            "text": "查询明天上海天气"
        }
    ],
    "bot_setting": [
        {
            "bot_name": "MM助手",
            "content": "MM助手是minimax开发的AI助手"
        }
    ],
    "reply_constraints": {
        "sender_type": "BOT",
        "sender_name": "MM助手"
    },
    "plugins": [
        "plugin_web_search"
    ]
}